In [1]:
# Install necessary libraries
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [2]:
# Import required libraries
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, AdamW
from torch.utils.data import DataLoader

In [3]:
# Load the Medical QnA Dataset
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# Define tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

In [14]:
dataset['train']['Question']

['Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'What are the symptoms of Lymphocytic Choriomeningitis (LCM) ?',
 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'How to diagnose Lymphocytic Choriomeningitis (LCM) ?',
 'What are the treatments for Lymphocytic Choriomeningitis (LCM) ?',
 'How to prevent Lymphocytic Choriomeningitis (LCM) ?',
 'What is (are) Parasites - Cysticercosis ?',
 'Who is at risk for Parasites - Cysticercosis? ?',
 'How to diagnose Parasites - Cysticercosis ?',
 'What are the treatments for Parasites - Cysticercosis ?',
 'How to prevent Parasites - Cysticercosis ?',
 'What is (are) Parasites - Trichuriasis (also known as Whipworm Infection) ?',
 'Who is at risk for Parasites - Trichuriasis (also known as Whipworm Infection)? ?',
 'How to diagnose Parasites - Trichuriasis (also known as Whipworm Infection) ?',
 'What are the treatments for Parasites - Trichuriasis (also known as Whipworm Infection) ?',
 'How to prevent Parasites - Trichu

In [6]:
# Data Preprocessing
def preprocess_data(example):
    return tokenizer(example['Question'], example['Answer'], truncation=True, padding='max_length', max_length=256)

train_data = dataset['train'].map(preprocess_data)
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)


Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

Streaming output truncated to the last 5000 lines.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned

In [13]:
train_loader

# Assuming you have a DataLoader named 'my_dataloader'
# for batch in train_loader:
#     print(batch)


In [ ]:
# Model Training
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):  # Adjust the number of epochs based on your needs
    for batch in train_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizer.step()



In [ ]:
# Evaluation
# Add code for evaluating the model on the validation set

# Save the trained model
model.save_pretrained("medical_qa_model")
tokenizer.save_pretrained("medical_qa_model")